In [ ]:
# Required for importing modules from parent directory
import os
import sys

current_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.dirname(current_dir)
print(parent_dir)
sys.path.append(parent_dir)

In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

ROOT = Path().cwd().parent
ROOT

In [ ]:
def load_log(path: Path):
    entries = []
    try:
        with open(path, "r") as file:
            for line in file:
                try:
                    log = json.loads(line.strip())
                    entries.append(log)
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON line: {e}")
    except Exception as e:
        print(f"Error reading JSON log file: {e}")
    return entries

In [ ]:
a2c = load_log(ROOT / "src/visualization/data/a2c_v1/logs_a2c.json")
ddpq = load_log(ROOT / "src/visualization/data/ddpg_v1/logs_ddpg.json")
dqn = load_log(ROOT / "src/visualization/data/dqn_v1/logs_dqn.json")
ppo = load_log(ROOT / "src/visualization/data/ppo_v1/logs_ppo.json")

len(a2c), len(ddpq), len(dqn), len(ppo)